In [1]:
import numpy as np
import sys
sys.path.append( '../vqe-in-dft' )
from tqdm import tqdm

import vqe_in_dft 

import scipy as sp
from pyscf import gto, dft, lib, mp, cc, scf, tools, ci, fci, lo, ao2mo

In [2]:
# geometry = [
# ['H', (0.7493682,0.0000000,0.4424329)],
# ['O', (0.0000000,0.0000000,-0.1653507)],
    
# ['H', (-0.7493682,0.0000000,0.4424329)]
# ]

geometry = [
['C', (0.0000, 0.0000, 0.0000)],
['H', (0.5288, 0.1610, 0.9359)],
['H', (0.2051, 0.8240, -0.6786)],
['H', (0.3345, -0.9314, -0.4496)],
['H', (-1.0685, -0.0537, 0.1921)]
]

# geometry = [
# ['H', (1.2473876659, -0.8998737590, 0.6150681570)],
# ['O', (1.2322305822, -0.2731895077, -0.1276123902)],

# ['C', (0.0849758188, 0.5590385475, 0.0510545434)],
# ['H', (0.1506137362, 1.1200249874, 0.9943015309)],
# ['H', (0.1316093068, 1.2841805400, -0.7645223601)],

# ['C', (-1.2129704155, -0.2295285634, -0.0097156258)],
# ['H', (-2.0801425360, 0.4329727646,0.0722817289)],
# ['H', (-1.2655910941, -0.9539857247, 0.8097953440)],
# ['H', (-1.2737541560, -0.7748626513, -0.9540587845)],
# ]

N_active_atoms = 2


low_level_scf_method='RKS'
high_level_scf_method='CCSD'
E_convergence_tol = 1e-6
basis = 'STO-3G' #'augccpvdz' # '6-31g' #'STO-3G'
unit= 'angstrom'
pyscf_print_level=1
memory=8000
charge=0
spin=0
run_fci= True#True
low_level_xc_functional = 'lda, vwn' # 'b3lyp'
high_level_xc_functional = 'b3lyp'

phys_notation = True

# Build the GLOBAL (full) molecule


and run supersystem calculation (cheap method)

https://pyscf.org/_modules/pyscf/gto/basis.html

In [3]:
full_system_mol = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       unit=unit,
                       spin=spin,
                      )
full_system_mol.build()
full_system_mol.atom

[['C', (0.0, 0.0, 0.0)],
 ['H', (0.5288, 0.161, 0.9359)],
 ['H', (0.2051, 0.824, -0.6786)],
 ['H', (0.3345, -0.9314, -0.4496)],
 ['H', (-1.0685, -0.0537, 0.1921)]]

In [4]:
## FCI comparison
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose=1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = fci.FCI(HF_scf).run()
print('E(UHF-FCI) = %.12f' % my_fci.e_tot)

# myci = ci.CISD(HF_scf).run() # this is UCISD
# print('UCISD total energy = ', myci.e_tot)

E(UHF-FCI) = -39.805676967111


In [5]:
HF_scf = scf.RHF(full_system_mol)
HF_scf.verbose= 1
HF_scf.max_memory= memory
HF_scf.conv_tol = 1e-6
HF_scf.kernel()

my_fci = ci.CISD(HF_scf).run()

E(RCISD) = -39.80303613703585  E_corr = -0.0762252226044578


In [6]:
# Draw molecule

xyz_string = vqe_in_dft.Get_xyz_string(full_system_mol)
vqe_in_dft.Draw_molecule(xyz_string, width=400, height=400, jupyter_notebook=True)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [7]:
# Run global calculation

full_system_scf = scf.RKS(full_system_mol)
full_system_scf.verbose= pyscf_print_level
full_system_scf.max_memory= memory
full_system_scf.conv_tol = 1e-6
full_system_scf.xc = low_level_xc_functional
full_system_scf.kernel()

-39.61684241519855

# Localise orbitals

### Background

The overlap matrix is:

$$S_{\mu \nu} =  \int d\vec{r}_{1} \phi_{\mu}(1)^{*}\phi_{\nu}(1)$$

- $\phi_{\mu}$ are basis functions (defined in basis set)


The unknown molecular orbitals $\psi_{i}$ are expanded as a linear expansion of the $K$ known basis functions:

$$ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$$


$C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$. The columns of $C$ describe the molecular orbitals!

We can find the total number of electrons $N$ in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  2 \sum_{a}^{N/2} 1$$

- integral gives probablity of finding electron $a$ over all space (must be 1)
- summing over all electrons will give the total number of electrons

The charge density has the following definition:

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg)$$

- re-write using definition of $\psi_{i}=  \sum_{\mu=1}^{K} C_{\mu i} \phi_{\mu}$

$$\rho(\vec{r}) = 2 \sum_{a}^{N/2} \Bigg( \bigg[ \sum_{\nu}^{K} C_{\nu a}^{*} \phi_{\nu}(\vec{r})^{*} \bigg] \bigg[ \sum_{\mu}^{K} C_{\mu a}\phi_{\mu}(\vec{r}) \bigg] \Bigg)$$

- move things around

$$\rho(\vec{r}) = \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$

- which is 

$$\rho(\vec{r}) = \sum_{\mu, \nu}^{K} P_{\mu \nu} \phi_{\mu}(\vec{r}) \phi_{\nu}(\vec{r})^{*} $$


- $P_{\mu \nu}$ is known as the density matrix and is:

$$P_{\mu \nu} = 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*}$$

Therefore we can also find the total number of electrons in the system by:

$$ N =  2 \sum_{a}^{N/2}\int d\vec{r}  \bigg( \psi_{a}(\vec{r})^{*} \psi_{i}(\vec{r}) \bigg) =  \sum_{\nu}^{K} \sum_{\mu}^{K} \Big( 2 \sum_{a}^{N/2} C_{\mu a} C_{\nu a}^{*} \Big) \int d\vec{r} \phi_{\mu}(\vec{r})  \phi_{\nu}(\vec{r})^{*}$$

- This is simply:

$$N =  \sum_{\nu}^{K} \sum_{\mu}^{K} P_{\mu \nu} S_{\nu \mu}= \sum_{\mu}^{K} PS_{\mu \mu} = \mathcal{Tr}(PS)$$

- One can interpret $ PS_{\mu \mu}$ in the above equation as the number of electrons associated with $ \phi_{\mu}$
- This is a **Mulliken population analysis**

# Orbital Localization!

A molecular orbital is usually delocalized, i.e. it has non-negligible amplitude over the whole system rather than only around some atom(s) or bond(s). However, one can choose a unitary rotation 

- When we perform a SCF calculation, one gets an optimized C matrix
    - $C$ is a $K \times K$ matrix of expansion coefficients $C_{\mu i}$
    - The columns of $C$ describe the molecular orbitals!
    - MO i: $ \psi_{i} =  \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu}$
    
    
- These molecular orbitals are usually **delocalized**
    - non-negligible amplitude over the whole system, rather than only around some atom(s) or bond(s)

- But we know in QM that a given basis choice is NOT unique


- We can therefore perform a unitary rotation on molecular orbitals

$$ \psi_{i} U_{rot} =  \Big( \sum_{\mu=1}^{K} C_{\mu i} \psi_{\mu} \Big) U_{rot} = \psi_{i}^{new}$$
    
    
The idea is to use a rotation such that the resulting orbitals $\psi_{i}^{new}$ are as spatially localized as possible. 


The Pipek-Mezey (PM) [localization](https://notendur.hi.is/hj/papers/paperPipekmezey8.pdf) **maximizes the population charges on the atoms**:

$$ f (U_{rot}) = \sum_{A}^{N_{atoms}} \Bigg( Z_{A} -  \sum_{\mu \text{ on atom } A} PS_{\mu \mu} \Bigg)$$

# Choose active and environment systems

## METHOD 1 

- Given optimized $C$ coefficient matrix
    - which has been rotated to localize orbitals
    - (used to build localized density matrix)


- **Look through basis functions $\phi_{\mu}$ of the ACTIVE atoms**

    
- check the mulliken charge // mulliken population of the orbital
    - if above a certain threshold associate it to active system
    - otherwise put in the environment
 


To choose the active and enviroment subsystems we do the following:

1. Given a localized molecular orbs (localized C matrix), we take the absolute mag squared of the coefficients of the active part for a given localized orb and divide by the absolute mag squared of all the coefficents of a that orb... THis will give a value of how much the active system contributes to that orb.

2. Mathematically, for orbital $j$ 
    - remember MO orbs given by columns of C matrix
    - In equation below C matrix is the LOCALIZED form!


$$ \text{threshold} =  \frac{\sum_{\mu\in \text{active AO}}^{K} |C_{\mu j}|^{2}}{\sum_{\mu =1}^{K} |C_{\mu j}|^{2}}$$

## METHOD 2 - SPADE

WRITE NOTES TODO

In [8]:
def Localize_orbitals(localization_method, PySCF_scf_obj, N_active_atoms, THRESHOLD=None, sanity_check=True):

    if PySCF_scf_obj.mo_coeff is None:
        raise ValueError('need to perform SCF calculation before localization')


    S_ovlp = PySCF_scf_obj.get_ovlp()
    AO_slice_matrix = PySCF_scf_obj.mol.aoslice_by_atom()

    occupied_orbs = PySCF_scf_obj.mo_coeff[:,PySCF_scf_obj.mo_occ>0]
    # run localization scheme
    if localization_method.lower() == 'spade':
        

        # Get active AO indices
        N_active_AO = AO_slice_matrix[N_active_atoms-1][3]  # find max AO index for active atoms (neg 1 as python indexs from 0)

        
        S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
        orthogonal_orbitals = (S_half@occupied_orbs)[:N_active_AO, :] # Get rows (the active AO) of orthogonal orbs 

        # Comute singular vals
        u, singular_values, rotation_matrix = np.linalg.svd(orthogonal_orbitals, full_matrices=True)

        # find where largest step change 
        delta_s = singular_values[:-1] - singular_values[1:] # σ_i - σ_(i+1)
        print('delta_singular_vals:')
        print(delta_s, '\n')

        n_act_mos = np.argmax(delta_s)+1 # add one due to python indexing
        n_env_mos = len(singular_values) - n_act_mos

        # define active and environment orbitals from localization
        act_orbitals = occupied_orbs @ rotation_matrix.T[:, :n_act_mos]
        env_orbitals = occupied_orbs @ rotation_matrix.T[:, n_act_mos:]

        C_matrix_all_localized_orbitals = occupied_orbs @ rotation_matrix.T

        active_MO_inds  = np.arange(n_act_mos)
        enviro_MO_inds = np.arange(n_act_mos, n_act_mos+n_env_mos)

    else:
        if not isinstance(THRESHOLD, float):
            raise ValueError ('if localization method is not SPADE then a threshold parameter is requried to choose active system')


        # run localization scheme
        if localization_method.lower() == 'pipekmezey':
            ### PipekMezey
            PM = lo.PipekMezey(PySCF_scf_obj.mol, occupied_orbs)
            PM.pop_method = 'mulliken' # 'meta-lowdin', 'iao', 'becke'
            C_loc_occ = PM.kernel() # includes virtual orbs too!

        elif localization_method.lower() == 'boys':
            ### Boys
            boys_SCF = lo.boys.Boys(PySCF_scf_obj.mol, occupied_orbs)
            C_loc_occ  = boys_SCF.kernel()

        elif localization_method.lower() == 'ibo':
            ### intrinsic bonding orbs
            #
            iaos = lo.iao.iao(PySCF_scf_obj.mol, occupied_orbs)
            # Orthogonalize IAO
            iaos = lo.vec_lowdin(iaos, S_ovlp)
            C_loc_occ = lo.ibo.ibo(PySCF_scf_obj.mol, occupied_orbs, locmethod='IBO', iaos=iaos)#.kernel()

            # iaos = lo.iao.iao(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff)
            # # Orthogonalize IAO
            # iaos = lo.vec_lowdin(iaos, S_ovlp)
            # C_loc = lo.ibo.ibo(PySCF_scf_obj.mol, PySCF_scf_obj.mo_coeff, locmethod='IBO', iaos=iaos)#.kernel()
            # C_loc_occ = C_loc[:,PySCF_scf_obj.mo_occ>0]
        else:
            raise ValueError(f'unknown localization method {localization_method}')
        
        
        S_half = sp.linalg.fractional_matrix_power(S_ovlp , 0.5)
        C_loc_occ_ORTHO = S_half@C_loc_occ 

        
        # find indices of AO of active atoms
        ao_active_inds = np.arange(AO_slice_matrix[0,2], AO_slice_matrix[N_active_atoms-1,3])

        ### New method
        numerator_all = np.einsum('ij->j', (C_loc_occ_ORTHO[ao_active_inds, :])**2) # active AOs coeffs for a given MO j
        denominator_all = np.einsum('ij->j', C_loc_occ_ORTHO**2) # all AOs coeffs for a given MO j

        MO_active_percentage = numerator_all/denominator_all

        print('\n(active_AO^2)/(all_AO^2):', np.around(MO_active_percentage,4))
        print(f'threshold for active part: {THRESHOLD} \n')

        active_MO_inds = np.where(MO_active_percentage>THRESHOLD)[0]
        enviro_MO_inds = np.array([i for i in range(C_loc_occ.shape[1]) if i not in active_MO_inds]) # get all non active MOs


        # define active MO orbs and environment
        act_orbitals = C_loc_occ[:, active_MO_inds] # take MO (columns of C_matrix) that have high dependence from active AOs
        env_orbitals = C_loc_occ[:, enviro_MO_inds]
        
        C_matrix_all_localized_orbitals = C_loc_occ

        n_act_mos = len(active_MO_inds)
        n_env_mos = len(enviro_MO_inds)

    print(f'number of active MOs: {n_act_mos}')
    print(f'number of enviro MOs: {n_env_mos} \n')

    return act_orbitals, env_orbitals, C_matrix_all_localized_orbitals, active_MO_inds, enviro_MO_inds # C_active, C_enviro, C_all_localized, active_MO_inds, enviro_MO_inds

In [9]:
localization_method= 'ibo' # 'ibo', 'Boys', 'PipekMezey' 'SPADE'
THRESHOLD = 0.95

(C_active, 
 C_envrio, 
 C_all_localized, 
 active_MO_inds,
 enviro_MO_inds) = Localize_orbitals(localization_method, 
                                     full_system_scf, 
                                     N_active_atoms, 
                                     THRESHOLD=THRESHOLD, 
                                     sanity_check=True)


 Iterative localization: IB/P4/2x2, 6 iter; Final gradient 9.22e-10

(active_AO^2)/(all_AO^2): [0.9982 0.5315 0.9998 0.5315 0.5315]
threshold for active part: 0.95 

number of active MOs: 2
number of enviro MOs: 3 



In [10]:
dm_loc = 2* C_all_localized@ C_all_localized.conj().T
dm_std = 2* full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]@ full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0].conj().T

print('want these C_loc and C_opt_std to be different:', not np.allclose(C_all_localized,
                                                                         full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]))
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc))

want these C_loc and C_opt_std to be different: True
want dm_std and dm_loc to be the SAME: True


Virtual localized orbs!:
https://pyscf.org/_modules/pyscf/lo/vvo.html

In [11]:
from pyscf.lo import vvo

orbocc = full_system_scf.mo_coeff[:,full_system_scf.mo_occ>0]
orbvirt = full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]
C_virtual_loc = vvo.vvo(full_system_scf.mol,
                    orbocc,
                    orbvirt,
                    iaos=None, 
                    s=None,
                    verbose=None)

C_virtual_loc.shape

# could truncate here using active MO again!!!

# PUT localized virtual orbs into full C matrix!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     C_virtual_loc))

In [12]:
# PUT standard virtual orbs into full C matrix (no change from standard HF virutal orbs)!
C_all_localized_and_virt = np.hstack((C_all_localized,
                                     full_system_scf.mo_coeff[:,full_system_scf.mo_occ<2]))

In [13]:
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
test =  S_half@ C_virtual_loc
test[:,0].dot(test[:,1])


5.412337245047638e-16

# IMPORTANT orbital localization point!

When we performed orbital localization a unitary rotation was used... this left the generated density matrix (C * C) unchanged...

HOWEVER we need to also **rotate the Fock matrix** otherwise the orbital energies no longer make any sense (even though the final energy will be the same)!


In [14]:
# here we see the localized and standard density matrices are the same!
print('want dm_std and dm_loc to be the SAME:',np.allclose(dm_std, dm_loc), '\n')


print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ full_system_scf.get_fock() @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

# NOTE HOW THERE IS A DIFFERENCE WHEN LOCALIZED... To fix this we need to rotate the Fock matrix!

want dm_std and dm_loc to be the SAME: True 

standard orb energies : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies : 

[-9.45464412 -0.40066658 -0.40069226 -0.40067845 -0.4006642   0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]


How to get the operator that changes the basis?

1. Put the standard (unlocalized) and localized C matrix into orthogonal basis
    - $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
    - $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

2. Define change of basis
    - $U = \sum_{i \in occupied} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, loc}} | + \sum_{i \in virtual} | \psi_{i}^{\text{ORTHO, STD}} \rangle \langle \psi_{i}^{\text{ortho, STD}}| $ 
    - note this projects occupied standard ortho orbs onto localized orbs
    - keeps virtual orbs the same
        - if we localize the virtual orbs then we would need to use the same procedure as the occupied orbs (easy to include)

In [15]:
# note can make this much better with einsum!

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

print(f'is U*C_ortho_loc =  C_ortho_STD: {(np.allclose(U @ ortho_loc, ortho_std))}')
print(f'is U Udag = Identity: {np.allclose(U.conj().T@U, np.eye(U.shape[0]))}')
        
# einsum version!
np.allclose(np.einsum('ik,jk->ij', ortho_std,ortho_loc),
            U)
    
    
# ind = 0
# Fock = full_system_scf.get_fock(dm=dm_std)
# print(ortho_loc[:,ind].conj().T @ (U.conj().T @ Fock @ U) @ ortho_loc[:,ind])
# print(ortho_std[:,ind].conj().T  @ Fock @ ortho_std[:,ind])

is U*C_ortho_loc =  C_ortho_STD: True
is U Udag = Identity: True


True

Note that $U$ is currently defined using ORTHOGONAL BASIS... we want it in our usual MO basis

Remembering that

$$C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$$

Clearly to get back to standard basis we do:

$$C_{\text{std}} = S^{-0.5} C_{\text{std}}^{\text{ORTHO}}$$


NOW as 


$$U C_{\text{loc}}^{\text{ORTHO}} =C_{\text{std}}^{\text{ORTHO}}$$

we can re-write this using our definitions:
- $C_{\text{std}}^{\text{ORTHO}} = S^{0.5}C_{\text{std}}$
- $C_{\text{loc}}^{\text{ORTHO}} = S^{0.5}C_{\text{loc}}$

$$U (S^{0.5}C_{\text{std}}) = (S^{0.5}C_{\text{loc}})$$

Now we want to get rid of $S^{0.5}$ on RHS so we times on left by $S^{-0.5}$

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = C_{\text{loc}}$$

Therefore the change of basis operator in our non-ortho basis is simply:

$$ S^{-0.5} U S^{0.5}C_{\text{std}} = V$$

Where: $ V C_{\text{std}} = C_{\text{loc}}$


In [16]:
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

V = S_neg_half @ U @ S_half


print('V @ C_loc = C_std:', np.allclose(V@C_all_localized_and_virt,
                                            full_system_scf.mo_coeff))

V @ C_loc = C_std: True


NOW we can check orbital energies of localized system when we use V to change basis:

In [17]:
print('standard orb energies : \n')
print(np.diag(full_system_scf.mo_coeff.conj().T @ full_system_scf.get_fock() @ full_system_scf.mo_coeff))
print(full_system_scf.mo_energy)

print('\nLOCALIZED orb energies WITHOUT Fock change of basis : \n')
print(np.diag(C_all_localized.conj().T @ full_system_scf.get_fock() @ C_all_localized))
print(full_system_scf.mo_energy)


print('\nLOCALIZED orb energies WITH Fock change of basis : \n')
print(np.diag(C_all_localized_and_virt.conj().T @ (V.conj().T @ full_system_scf.get_fock() @ V) @ C_all_localized_and_virt))
print(full_system_scf.mo_energy)

standard orb energies : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies WITHOUT Fock change of basis : 

[-9.45464412 -0.40066658 -0.40069226 -0.40067845 -0.4006642 ]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]

LOCALIZED orb energies WITH Fock change of basis : 

[-9.56763009 -0.58148699 -0.30276035 -0.3027371  -0.30273108  0.4253797
  0.42539908  0.42544391  0.47125932]
[-9.56762769 -0.58148626 -0.30275977 -0.30273646 -0.30273044  0.42538041
  0.4253998   0.42544456  0.4712599 ]


In [18]:
# modify H_core and Veff to allow this change of basis!!!

def Get_new_Hcore(H_core, Unitary_rot):
    H_core_rot = Unitary_rot.conj().T @ H_core @Unitary_rot 
    return H_core_rot

from pyscf.dft.rks import get_veff as RKS_get_veff
## RKS_get_veff gives Coulomb + XC functional
## in matrix form: Veff = J + Vxc.
    
from pyscf import lib
from pyscf.dft import numint

def Get_new_Veff(pyscf_obj, Unitary, dm=None, check_result=False):
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    
#     Evaluate RKS/UKS XC functional and potential matrix on given meshgrids
#     for a set of density matrices.
    nelec, exc, vxc = numint.nr_vxc(pyscf_obj.mol,
                                            pyscf_obj.grids,
                                            pyscf_obj.xc,
                                            density_mat)
    vxc =  Unitary.conj().T @ vxc @ Unitary
    
    
    Veff = RKS_get_veff(pyscf_obj, dm=density_mat)
    if Veff.vk is not None:
        K = Unitary.conj().T @ Veff.vk @ Unitary
        J = Unitary.conj().T @ Veff.vj @ Unitary
        vxc += J - K * .5
    else:
        J = Unitary.conj().T @ Veff.vj @ Unitary
        K = None
        vxc += J 
    
    if check_result is True:
        M1 = Unitary.conj().T @ Veff.__array__() @ Unitary
        if not np.allclose(vxc, M1):
            raise ValueError('Veff in new basis NOT correct')
    
    ecoul = np.einsum('ij,ji', density_mat, J).real * .5 # note J matrix is in new basis!
    ## this ecoul term changes if the full density matrix is NOT 
    # (aka for dm_active and dm_enviroment we get different V_eff under different bases!)
    
    
    output = lib.tag_array(vxc, ecoul=ecoul, exc=Veff.exc, vj=J, vk=K)
    return output


In [19]:
# Check pyscf obj with localized orbs and unitary change of basis gives same results as before!

test_full_system_scf = scf.RKS(full_system_mol)
test_full_system_scf.verbose= pyscf_print_level
test_full_system_scf.max_memory= memory
test_full_system_scf.conv_tol = 1e-6
test_full_system_scf.xc = low_level_xc_functional


Hcore_std = test_full_system_scf.get_hcore()
test_full_system_scf.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


test_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    test_full_system_scf,
                                                                                       V,
                                                                                        dm=dm)


test_full_system_scf.mo_coeff = C_all_localized_and_virt
test_full_system_scf.mo_occ = full_system_scf.mo_occ

test_full_system_scf.energy_tot(dm=dm_loc)

-39.61684241524395

In [20]:
C_std = full_system_scf.mo_coeff

print(np.allclose(C_std.conj().T @ full_system_scf.get_hcore() @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_hcore() @ C_all_localized_and_virt))

print(np.allclose(C_std.conj().T @ full_system_scf.get_veff(dm=dm_std) @ C_std,
           C_all_localized_and_virt.conj().T @ test_full_system_scf.get_veff(dm=dm_loc) @ C_all_localized_and_virt))

True
True


In [21]:
### RKS energy (not the same as HFock!)
vhf_std = full_system_scf.get_veff(dm=dm_std)
e1_std = np.einsum('ij,ji->', full_system_scf.get_hcore(), dm_std)
e2_std = vhf_std.ecoul + vhf_std.exc
print(e1_std + e2_std)

print()
vhf_loc = test_full_system_scf.get_veff(dm=dm_std)
e1_loc = np.einsum('ij,ji->', test_full_system_scf.get_hcore(), dm_loc)
e2_loc = vhf_loc.ecoul + vhf_loc.exc
print(e1_loc+e2_loc)

-53.08968539169484

-53.089685391740275


In [22]:
print('check electronic energies match:', np.allclose(full_system_scf.energy_elec(dm=dm_std)[0],
                                test_full_system_scf.energy_elec(dm=dm_loc)[0]))

# test_full_system_scf.kernel()  # run to populate mo_energy....
# print('check MO energies match:', np.allclose(full_system_scf.mo_energy,
#                                 test_full_system_scf.mo_energy))

check electronic energies match: True


In [23]:
F_new = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm_loc) # Note this is now in new basis!
print(np.around(C_all_localized_and_virt.conj().T @ F_new @ C_all_localized_and_virt, 2))


F_std = full_system_scf.get_hcore() + full_system_scf.get_veff(dm=dm_loc) # Note this is standard HF!
C_std = full_system_scf.mo_coeff
print(np.around(C_std.conj().T @ F_std @ C_std, 2))

[[-9.57  0.    0.   -0.    0.    0.   -0.   -0.    0.  ]
 [ 0.   -0.58  0.   -0.    0.   -0.    0.    0.   -0.  ]
 [ 0.    0.   -0.3  -0.    0.    0.    0.    0.   -0.  ]
 [-0.   -0.   -0.   -0.3   0.    0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.3  -0.   -0.    0.   -0.  ]
 [ 0.   -0.    0.    0.   -0.    0.43  0.   -0.    0.  ]
 [-0.    0.    0.   -0.   -0.    0.    0.43  0.   -0.  ]
 [-0.    0.    0.    0.    0.   -0.    0.    0.43 -0.  ]
 [ 0.   -0.   -0.    0.   -0.    0.   -0.   -0.    0.47]]
[[-9.57  0.    0.   -0.    0.    0.   -0.   -0.    0.  ]
 [ 0.   -0.58  0.   -0.    0.   -0.    0.    0.   -0.  ]
 [ 0.    0.   -0.3  -0.    0.    0.    0.    0.   -0.  ]
 [-0.   -0.   -0.   -0.3   0.    0.   -0.    0.    0.  ]
 [ 0.    0.    0.    0.   -0.3  -0.   -0.    0.   -0.  ]
 [ 0.   -0.    0.    0.   -0.    0.43  0.   -0.    0.  ]
 [-0.    0.    0.   -0.   -0.    0.    0.43  0.   -0.  ]
 [-0.    0.    0.    0.    0.   -0.    0.    0.43 -0.  ]
 [ 0.   -0.   -0.    0.   -0. 

Basically we now have orbital energies matching as we want

(This will become important when we move to post Hartree-Fock methods!)

In [24]:
# # Draw active orbitals

# list_active_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              active_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

# list_enviro_orbitals = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              C_all_localized, 
#                              enviro_MO_inds,
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [25]:
# list_active_orbitals[0]

In [26]:
# list_enviro_orbitals[0]

In [27]:
def Get_active_and_envrio_dm(PySCF_scf_obj, C_active, C_envrio, C_all_localized, sanity_check=True):

    # get density matrices
    dm_active =  2 * C_active @ C_active.T
    dm_enviro =  2 * C_envrio @ C_envrio.T

    if sanity_check:

        S_ovlp = PySCF_scf_obj.get_ovlp()

        ## check number of electrons is still the same after orbitals have been localized (change of basis)
        N_active_electrons = np.trace(dm_active@S_ovlp)
        N_enviro_electrons = np.trace(dm_enviro@S_ovlp)
        N_all_electrons = PySCF_scf_obj.mol.nelectron

        bool_flag_electron_number = np.isclose(( N_active_electrons + N_enviro_electrons), N_all_electrons)
        if not bool_flag_electron_number:
            raise ValueError('number of electrons in localized orbitals is incorrect')
        print(f'N_active_elec + N_environment_elec = N_total_elec is: {bool_flag_electron_number}')

        # checking denisty matrix parition makes sense:
        dm_localised_full_system = 2* C_all_localized@ C_all_localized.conj().T
        bool_density_flag = np.allclose(dm_localised_full_system, dm_active + dm_enviro)
        if not bool_density_flag:
            raise ValueError('gamma_full != gamma_active + gamma_enviro')
        print(f'y_active + y_enviro = y_total is: {bool_density_flag}')

    return dm_active, dm_enviro

In [28]:
dm_active, dm_enviro = Get_active_and_envrio_dm(
                                            full_system_scf,
                                            C_active, 
                                            C_envrio, 
                                            C_all_localized, 
                                            sanity_check=True)

N_active_elec + N_environment_elec = N_total_elec is: True
y_active + y_enviro = y_total is: True


In [29]:
C_envrio_ORTHO = S_half @ C_envrio
dm_env_ORTHO = 2* C_envrio_ORTHO@C_envrio_ORTHO.conj().T

PB = 0.5* np.eye(S_mat.shape[0])@ dm_env_ORTHO @ np.eye(S_mat.shape[0])

# S_mat_mod = V.conj().T @ S_mat @ V
# PB = S_mat_mod @ dm_enviro @ S_mat_mod

print(np.allclose(PB @ C_envrio_ORTHO[:,0], C_envrio_ORTHO[:,0]))
# ^ works in orthogonal basis!!!!!


S_mat_mod = V.conj().T @ S_mat @ V
PB = S_mat_mod @ dm_enviro @ S_mat_mod
print(np.allclose(PB @ C_envrio[:,0], C_envrio[:,0]))
# ^ doesn't work in non ortho basis

True
False


In [30]:
from pyscf.lo import orth, nao
# orth_coeff = orth.orth_ao(full_system_scf, 'meta_lowdin', s=S_mat)

# orth_coeff.shape
# # np.allclose(orth_coeff, C_envrio_ORTHO)

c0 = nao.prenao(full_system_scf.mol, dm_enviro)
c =orth.orth_ao(full_system_scf, 'NAO', c0)
c

array([[ 9.72059317e-01, -3.89212634e-01,  1.39173447e-06,
         1.08722493e-06,  1.38746467e-06, -1.93112183e-03,
        -1.92778039e-03, -1.92771547e-03, -1.92989364e-03],
       [ 9.53050117e-02,  1.55614348e+00, -1.90315635e-05,
        -1.19406028e-05, -5.54175704e-06, -3.86793432e-01,
        -3.86766155e-01, -3.86768365e-01, -3.86789003e-01],
       [-6.12541300e-16, -1.58397203e-05,  1.17372833e+00,
        -1.89004191e-06,  1.48254737e-05, -1.72132757e-01,
        -6.67564832e-02, -1.08875038e-01,  3.47803962e-01],
       [-3.84985451e-16, -9.95534807e-06, -1.89004191e-06,
         1.17370830e+00, -2.05082170e-06, -5.24075550e-02,
        -2.68198800e-01,  3.03156572e-01,  1.74794222e-02],
       [-1.81441894e-16, -4.69191033e-06,  1.48254737e-05,
        -2.05082170e-06,  1.17374341e+00, -3.04649952e-01,
         2.20874333e-01,  1.46338372e-01, -6.25293813e-02],
       [-1.22257880e-11, -3.16146946e-01, -1.70339632e-01,
        -5.18616200e-02, -3.01476381e-01,  1.283731

In [31]:
PB@C_envrio[:,0]
C_envrio[:,0]

array([-0.05786098,  0.31740174,  0.09641271,  0.38725087, -0.31887367,
       -0.05716423,  0.52474686, -0.05717071, -0.05717224])

In [32]:
two_e_term =  test_full_system_scf.get_veff(dm=dm_loc)

np.allclose(V.conj().T @test_full_system_scf.get_j(dm = dm_loc) @ V,
            two_e_term.vj)

True

In [33]:
def Get_energy_and_matrices_from_dm(PySCF_scf_obj, dm_matrix, check_E_with_pyscf=True):
    """
    Get Energy from denisty matrix

    Note this uses the standard hcore (NO embedding potential here!)
    """

    # It seems that PySCF lumps J and K in the J array 
    two_e_term =  PySCF_scf_obj.get_veff(dm=dm_matrix)
    J_mat = two_e_term.vj
    K_mat = np.zeros_like(J_mat)
    
    e_xc = two_e_term.exc
    v_xc = two_e_term - J_mat 

#     H_core_standard = scf.hf.get_hcore(PySCF_scf_obj.mol) # No embedding potential
    Energy_elec = (np.einsum('ij,ji->', PySCF_scf_obj.get_hcore(), dm_matrix) + 
                   two_e_term.ecoul + two_e_term.exc)
    
    if check_E_with_pyscf:
        Energy_elec_pyscf = PySCF_scf_obj.energy_elec(dm=dm_matrix)[0]
        if not np.isclose(Energy_elec_pyscf, Energy_elec):
            raise ValueError('Energy calculation incorrect')

    return Energy_elec, J_mat, K_mat, e_xc, v_xc

In [34]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( full_system_scf, # <-- standard
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-42.16618474131516


-42.16618474131516

In [35]:
# use active density
E_act, J_act, K_act, e_xc_act, v_xc_act = Get_energy_and_matrices_from_dm( test_full_system_scf, # <-- modified 
                                                                             dm_active, # <- ACTIVE
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_active)[0])
E_act

-42.34822544673901


-42.34822544673901

In [36]:
## Note different energies!
print(full_system_scf.energy_elec(dm=dm_active)[0])
print(test_full_system_scf.energy_elec(dm=dm_active)[0])

## This is expected as now using active density matrix (rather than full one)

print()
# If we use the full dm matrix then we get matching results (aka now have full density matrix)
E_loc = test_full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0]
E_std = full_system_scf.energy_elec(dm=dm_active+dm_enviro)[0] 
print(E_loc)
print(E_std)

-42.16618474131516
-42.34822544673901

-53.08968539174023
-53.089685391694864


In [37]:
# Note we want to use modified object, where change of basis on Fock matrix has been performed!

In [38]:
# use enviro density
E_env, J_env, K_env, e_xc_env, v_xc_env = Get_energy_and_matrices_from_dm(
                                                                            test_full_system_scf, # <-- modified 
                                                                             dm_enviro, # <- ENVIRO
                                                                             check_E_with_pyscf=True)
print(test_full_system_scf.energy_elec(dm=dm_enviro)[0])
E_env

-23.778806912773213


-23.778806912773213

In [39]:
def Get_cross_terms(PySCF_scf_obj, dm_active, dm_enviro, J_env, J_act, e_xc_act, e_xc_env):
    """
    Get cross system terms
    
    """

    two_e_term_total =  PySCF_scf_obj.get_veff(dm=dm_active+dm_enviro)
    e_xc_total = two_e_term_total.exc

    j_cross = 0.5 * ( np.einsum('ij,ij', dm_active, J_env) + np.einsum('ij,ij', dm_enviro, J_act) )
    k_cross = 0.0

    xc_cross = e_xc_total - e_xc_act - e_xc_env
    two_e_cross = j_cross + k_cross + xc_cross
    
    return two_e_cross

In [40]:
# cross terms!
two_e_cross = Get_cross_terms(test_full_system_scf, # <-- modified 
                                         dm_active, 
                                         dm_enviro, 
                                         J_env, 
                                         J_act, 
                                         e_xc_act,
                                         e_xc_env)

In [41]:
print(f'E_active: {E_act}')
print(f'E_enviro: {E_env}')
print(f'E_cross: {two_e_cross}')

E_active: -42.34822544673901
E_enviro: -23.778806912773213
E_cross: 13.037346967772002


In [42]:
print(f'E_hf_standard : {full_system_scf.e_tot}')
print(f'E_act + E_env + non_add_two_e + E_nuc : {E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()}')

# equation 1 of SPADE paper
print(f'energies match {np.isclose(full_system_scf.e_tot, (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc()))}') # equation 3 of SPADE paper
full_system_scf.e_tot - (E_act+E_env+two_e_cross+ full_system_scf.energy_nuc())

E_hf_standard : -39.61684241519855
E_act + E_env + non_add_two_e + E_nuc : -39.616842415243916
energies match True


4.53681536782824e-11

In [43]:
print(np.trace(S_mat@dm_enviro))
print(np.trace(S_mat@dm_active))

6.000000000000007
4.000000000000002


# NEXT define the projector onto subsystem B (enviroment)

$$P^{env} = \sum_{i \in \text{env}} |\psi_{i} \rangle \langle \psi_{i}|$$

important points:
1. Currently C_localized matrix columns give MO orbs $|\psi_{i} \rangle$
2. However, these are **not orthogonal**! Therefore defining a projector here is HARD
3. To fix this:
    - change to orthogonal basis
    - now define projector in this new basis
    - undo basis change to give projector in non-ortho basis!

In [44]:
print(sum(np.abs(C_all_localized_and_virt[:,0])**2))

Loc_Ortho = S_half@ C_all_localized_and_virt 
print(sum(np.abs(Loc_Ortho[:,0])**2))

1.0198275945611663
0.9999999999999962


In [45]:
PB_ortho = np.einsum('ik,jk->ij', Loc_Ortho[:, enviro_MO_inds], Loc_Ortho[:, enviro_MO_inds])

C_envrio_ORTHO = S_half @ C_envrio
dm_env_ORTHO = 2* C_envrio_ORTHO@C_envrio_ORTHO.conj().T


# MU SHIFT APPROACH!
PB = 0.5* np.eye(S_mat.shape[0])@ dm_env_ORTHO @ np.eye(S_mat.shape[0])
np.allclose(PB_ortho, PB)

True

In [46]:
print(np.allclose(PB_ortho@C_envrio_ORTHO, C_envrio_ORTHO))

True


In [47]:
ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt
U_ortho_changaee_basis = np.einsum('ik,jk->ij', ortho_std,ortho_loc)

In [48]:
mu = 1e6
MU_PB_ortho = mu* np.einsum('ik,jk->ij', Loc_Ortho[:, enviro_MO_inds], Loc_Ortho[:, enviro_MO_inds])


PB_ortho = np.einsum('ik,jk->ij', Loc_Ortho[:, enviro_MO_inds], Loc_Ortho[:, enviro_MO_inds])

In [49]:
#alg

full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED_HF) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6


# guess Dmat
H_core = EMBEDDED_full_system_scf_HF.get_hcore()

mo_energy, mo_coeff_ORTHO = np.linalg.eigh(H_core) # Fock guess as Hcore!
mo_coeff = S_neg_half @ mo_coeff_ORTHO
mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)
D = EMBEDDED_full_system_scf_HF.make_rdm1(mo_coeff=mo_coeff, mo_occ=mo_occ)

max_iter = 100
HF_energy =0
E_previous = 0
E_tol = 1e-6

for i in range(max_iter+1):
    
    # Build Fock Matrix
    
    vhf = EMBEDDED_full_system_scf_HF.get_veff(dm=D)
    
    ### find RHF energy
    e1 = np.einsum('ij,ji->', H_core, D)
    e_coul = np.einsum('ij,ji->', vhf, D) * .5
    e_proj = np.einsum('ij,ji->', Proj_NORMAL, D)
    HF_energy = e1 + e_coul + EMBEDDED_full_system_scf_HF.energy_nuc() + e_proj
#     HF_energy =np.trace(H_core @ D) + np.trace(Fock @ D) + full_system_mol.energy_nuc()
    # (no 0.5 here as included in D mat natively!)
    
    ### check convergence
    if np.abs(HF_energy-E_previous)<E_tol:
        break
    
    
#     fock_ortho =  S_neg_half@ (vhf + H_core)@ S_neg_half + MU_PB_ortho
# #     Proj_NORMAL =  S_neg_half @ MU_PB_ortho @ S_half
#     Proj_NORMAL =  S_half @ MU_PB_ortho @ S_half
#     print(np.allclose(np.diag(C_envrio.conj().T@ Proj_NORMAL@C_envrio),
#            mu*np.ones(C_envrio.shape[1]))) # shows projection onto high energy state!
    
    fock_ortho = S_neg_half@ (vhf + H_core)@ S_neg_half 
    huz = -1*(fock_ortho@PB_ortho + PB_ortho@fock_ortho)
    huz2 = -0.5*(fock_ortho@dm_env_ORTHO + dm_env_ORTHO@fock_ortho)
    print(np.allclose(huz, huz2))
#     Proj_NORMAL =  S_neg_half @ huz @ S_half
    Proj_NORMAL =  S_half @ huz @ S_half
    fock_ortho += huz # huz in ortho basis too!
    

#     fock_DFT = test_full_system_scf.get_fock(dm=dm_active+dm_enviro)
#     fock_DFT_ortho = S_neg_half@ fock_DFT @ S_neg_half 
#     huz = -1*(fock_DFT_ortho@PB_ortho + PB_ortho@fock_DFT_ortho)/2
#     Proj_NORMAL =  S_half @ huz @ S_half
#     fock_ortho =  S_neg_half@ (vhf + H_core + Proj_NORMAL)@ S_neg_half
#     print(np.allclose(C_envrio_ORTHO.conj().T @huz @ C_envrio_ORTHO,
#            -1*C_envrio_ORTHO.conj().T @fock_DFT_ortho @ C_envrio_ORTHO))
#     print(np.allclose(C_envrio.conj().T @ (Proj_NORMAL) @ C_envrio,
#             -1*C_envrio.conj().T @fock_DFT @ C_envrio))


    mo_energy, mo_coeff_ORTHO = np.linalg.eigh(fock_ortho)
    
    mo_coeff = S_neg_half @ mo_coeff_ORTHO
    mo_occ = EMBEDDED_full_system_scf_HF.get_occ(mo_energy, mo_coeff)
    D = EMBEDDED_full_system_scf_HF.make_rdm1(mo_coeff=mo_coeff, mo_occ=mo_occ)
    
    
    # if not convereged store old result
    E_previous = HF_energy
    
    if i==max_iter:
        raise ValueError('Maximum number of SCF iterations exceeded')
        
print(f'final RHF SCF energy {HF_energy}')

NameError: name 'Proj_NORMAL' is not defined

In [ ]:
# # for Huzianga
# fock_DFT = test_full_system_scf.get_fock(dm=dm_active+dm_enviro)
# fock_DFT_ortho = S_neg_half@ fock_DFT @ S_neg_half 
# huz = -1*(fock_DFT_ortho@PB_ortho + PB_ortho@fock_DFT_ortho)/2
# Proj_NORMAL =  S_half @ huz @ S_half

# print(np.allclose(C_envrio.conj().T @ (fock_DFT + Proj_NORMAL) @ C_envrio,
#         np.zeros_like(C_envrio.shape[1]))) # zeros out values!


# EMBEDDED_full_system_scf_HF.mo_coeff = mo_coeff
# EMBEDDED_full_system_scf_HF.mo_occ = mo_occ

# mo_energy = np.diag( mo_coeff.conj().T @ EMBEDDED_full_system_scf_HF.get_fock() @ mo_coeff)

In [ ]:
mo_energy

In [ ]:
mu_shfit_orb_E = mo_energy[:-len(enviro_MO_inds)]
mu_shfit_orb_E

In [ ]:
huz_orb_E = mo_energy[:-len(enviro_MO_inds)]
huz_orb_E

In [ ]:
mu_shfit_orb_E - huz_orb_E

In [ ]:
# for mu shift
print(np.allclose(np.diag(C_envrio.conj().T@ Proj_NORMAL@C_envrio),
           mu*np.ones(C_envrio.shape[1])))

In [ ]:
EMBEDDED_full_system_scf_HF.mo_coeff = mo_coeff
EMBEDDED_full_system_scf_HF.mo_occ = mo_occ

np.diag( mo_coeff.conj().T @ EMBEDDED_full_system_scf_HF.get_fock() @ mo_coeff)



In [ ]:
EMBEDDED_full_system_scf_HF.mo_coeff = mo_coeff
EMBEDDED_full_system_scf_HF.mo_occ = mo_occ

H_core_std = EMBEDDED_full_system_scf_HF.get_hcore()
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: H_core_std + Proj_NORMAL # add projector here!


In [ ]:
E_pyscf = EMBEDDED_full_system_scf_HF.energy_elec()[0] + EMBEDDED_full_system_scf_HF.energy_nuc() 

np.isclose(E_pyscf, HF_energy)

In [ ]:
# np.diag( mo_coeff.conj().T @ EMBEDDED_full_system_scf_HF.get_fock() @ mo_coeff)

In [ ]:
-29.657814422987833 + 29.767702716211193

In [ ]:
Proj_NORMAL = S_half  @ huz @ S_neg_half

# print(np.allclose( Proj_NORMAL @ C_envrio,
#     C_envrio))

Proj_NORMAL*2 @ C_envrio

# C_envrio

In [ ]:
print(np.allclose(PB_ortho@C_envrio_ORTHO, C_envrio_ORTHO))

Proj_NORMAL =  S_neg_half @ PB_ortho @ S_half
print(np.allclose(PB_ortho@C_envrio_ORTHO, C_envrio_ORTHO))

In [ ]:
Proj_NORMAL =  S_half @ PB_ortho @ S_half
np.allclose(Proj_NORMAL@ C_envrio, C_envrio)

Proj_NORMAL@ C_envrio

C_envrio

In [ ]:
mu *(C_envrio.conj().T @fock_DFT @ C_envrio)

In [ ]:
fock_DFT = test_full_system_scf.get_fock(dm=dm_active+dm_enviro)
fock_DFT_ortho = S_neg_half@ fock_DFT @ S_neg_half 
huz = -1*(fock_DFT_ortho@PB_ortho + PB_ortho@fock_DFT_ortho)/2
Proj_NORMAL =  S_half @ huz @ S_half

In [ ]:
np.around(C_envrio.conj().T @fock_DFT @ C_envrio,3)
np.around(C_envrio.conj().T @(Proj_NORMAL) @ C_envrio,3)


C_envrio.conj().T @(fock_DFT+Proj_NORMAL) @ C_envrio 

In [ ]:
fock_DFT = test_full_system_scf.get_fock(dm=dm_active+dm_enviro)
fock_DFT_ortho = S_neg_half@ fock_DFT @ S_neg_half 
huz = -1*(fock_DFT_ortho@PB_ortho + PB_ortho@fock_DFT_ortho)/2
Proj_NORMAL =  S_half @ huz @ S_half
fock_ortho =  S_neg_half@ (vhf + H_core + Proj_NORMAL)@ S_neg_half


print(np.allclose(C_envrio_ORTHO.conj().T @huz @ C_envrio_ORTHO,
       -1*C_envrio_ORTHO.conj().T @fock_DFT_ortho @ C_envrio_ORTHO))
print(np.allclose(C_envrio.conj().T @Proj_NORMAL @ C_envrio_ORTHO,
       -1*C_envrio.conj().T @fock_DFT @ C_envrio))

In [ ]:
np.einsum('ij,ji->', Proj_NORMAL, D)

In [ ]:
# huz = -1*(fock_DFT_ortho@PB_ortho + PB_ortho@fock_DFT_ortho)/2
# Proj_NORMAL =  S_half @ huz @ S_half

np.around((C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ Proj_NORMAL
           @ C_all_localized_and_virt[:, enviro_MO_inds]),3)

np.around((C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ (fock_DFT)
           @ C_all_localized_and_virt[:, enviro_MO_inds]),3)

# np.around((C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ (fock_DFT+Proj_NORMAL)
#            @ C_all_localized_and_virt[:, enviro_MO_inds]),3)

In [ ]:
Proj_NORMAL =  S_half @ PB_ortho @ S_half

np.around((C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ Proj_NORMAL
           @ C_all_localized_and_virt[:, enviro_MO_inds]),20)

In [ ]:
Proj_NORMAL =  S_neg_half @ huz @ S_half
np.around((C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ Proj_NORMAL
           @ C_all_localized_and_virt[:, enviro_MO_inds]),3)

In [ ]:
# Proj_NORMAL =  S_half @ huz @ S_half
# np.einsum('ij,ji->', Proj_NORMAL, D)

Proj_NORMAL =  S_half @ MU_PB_ortho @ S_half
np.einsum('ij,ji->', Proj_NORMAL, D)

print(np.allclose(Proj_NORMAL@C_all_localized_and_virt[:, enviro_MO_inds],
           mu*C_all_localized_and_virt[:, enviro_MO_inds]) # <-- note mu here!
)

In [ ]:
Proj_NORMAL =  S_half @ MU_PB_ortho @ S_half
# Proj_NORMAL =  S_neg_half @ MU_PB_ortho @ S_half
C_all_localized_and_virt[:, enviro_MO_inds].conj().T @ Proj_NORMAL@C_all_localized_and_virt[:, enviro_MO_inds]

In [ ]:
# huz = -1*(fock_ortho@PB_ortho + PB_ortho@fock_ortho)/2
# Proj_NORMAL =  S_half @ huz @ S_half

# print(np.allclose(C_envrio.conj().T @Proj_NORMAL @ C_envrio,
#            -1*C_envrio.conj().T @fock_DFT @ C_envrio))

C_envrio.conj().T @Proj_NORMAL @ C_envrio

In [ ]:
print(np.allclose(Proj_NORMAL@C_all_localized_and_virt[:, enviro_MO_inds],
           -1*C_all_localized_and_virt[:, enviro_MO_inds]) 
)


In [ ]:
# # mu_shift!
# print(np.allclose(Proj_NORMAL@C_all_localized_and_virt[:, enviro_MO_inds],
#            mu*C_all_localized_and_virt[:, enviro_MO_inds]) # <-- note mu here!
# )

# # huzinaga!
fock_DFT = test_full_system_scf.get_fock(dm=dm_active+dm_enviro)
fock_ortho = S_neg_half@ fock_DFT @ S_neg_half 

huz = -1*(fock_ortho@PB_ortho + PB_ortho@fock_ortho)/2
huz2 = -0.5*(fock_ortho@dm_env_ORTHO + dm_env_ORTHO@fock_ortho)/2


print(np.allclose(C_envrio_ORTHO.conj().T @huz @ C_envrio_ORTHO,
           -1*C_envrio_ORTHO.conj().T @fock_ortho @ C_envrio_ORTHO))

Proj_NORMAL =  S_half @ huz @ S_half
print(np.allclose(C_envrio.conj().T @Proj_NORMAL @ C_envrio,
           -1*C_envrio.conj().T @fock_DFT @ C_envrio))

In [ ]:
print(np.around(C_envrio.conj().T @Proj_NORMAL @ C_envrio,3))

np.around(C_envrio.conj().T @fock_DFT @ C_envrio,3)

In [ ]:
Proj_NORMAL =  S_neg_half @ huz @ S_half

np.allclose(C_envrio.conj().T @Proj_NORMAL @ C_envrio,
           -1*C_envrio.conj().T @fock_DFT @ C_envrio)

In [ ]:
print(np.around(C_envrio_ORTHO.conj().T @huz @ C_envrio_ORTHO, 6))
np.around(C_envrio_ORTHO.conj().T @fock_ortho @ C_envrio_ORTHO, 6)

In [ ]:
print(np.around(C_envrio_ORTHO.conj().T @huz @ C_envrio_ORTHO, 6))
np.around(C_envrio_ORTHO.conj().T @fock_ortho @ C_envrio_ORTHO, 6)

print(np.einsum('ij,ji->', huz, dm_env_ORTHO))
np.einsum('ij,ji->', fock_ortho, dm_env_ORTHO)

In [ ]:
Proj_NORMAL@C_all_localized_and_virt[:, enviro_MO_inds]
C_all_localized_and_virt[:, enviro_MO_inds]

In [ ]:
PP = S_half@ PB_ortho @ S_half
e_PROJ = np.einsum('ij,ji->', PP, D)
e_PROJ

In [ ]:
np.around(mo_coeff_ORTHO.conj().T @ fock_ortho @ mo_coeff_ORTHO, 5)

In [ ]:
### CCSD calculation
projector_method = 'mu_shfit'
# projector_method = 'huzinaga'
HF_dm_mat = D

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

if projector_method == 'mu_shfit':
    embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
elif projector_method == 'huzinaga':
#     embedded_cc_obj.frozen  = enviro_MO_inds.tolist()
    embedded_cc_obj.frozen  = list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))
else:
    raise ValueError(f'unknown projector method {projector_method}')


# embedded_cc_obj.frozen = None
e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=HF_dm_mat),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf


In [ ]:
-29.400287715906902 + 29.400289626660157

In [ ]:
e_cc+embedded_cc_obj.e_hf

In [ ]:
EMBEDDED_full_system_scf_HF

In [ ]:
EMBEDDED_full_system_scf_HF.kernel()

In [ ]:
EMBEDDED_full_system_scf_HF

In [ ]:
mo_energy

In [ ]:
sdfads

In [ ]:
## manual projector

## 1. convert to orthogonal C_matrix
S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt # orthogonal C matrix (localized)

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
### not we only take MO environment indices!
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

PROJ_ortho_new = np.einsum('ik,jk->ij', Loc_Ortho[:, enviro_MO_inds], Loc_Ortho[:, enviro_MO_inds])
print(np.allclose(PROJ_ortho_new, PROJ_ortho))
        
print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!
        
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
        
        
projector_method = 'huzinaga'
# projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
    Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    # Huzinaga
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
#     projector = S_half @ projector_ortho  @ S_half
    
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector_ortho = mu * PROJ_ortho
    
#     projector = S_neg_half @ projector_ortho  @ S_half

    # mu shift
    projector = S_half @ projector_ortho  @ S_half
else:
    raise ValueError(f'Unknown projection method {projector_method}')

##### 4. Define projector in standard (non-orthogonal basis)
# projector = S_half @ projector_ortho  @ S_half


### 5. now can build relevant ops



### TODO check this!!!
# when using full_system_scf and mu_shift get correct result
# but I think one should use test_full_system_scf (with modified Fock matrix) 

### NOTE: we do NOT use modified Fock matrix here!!!!!
# g_A_and_B = full_system_scf.get_veff(dm=dm_active+dm_enviro)
# g_A = full_system_scf.get_veff(dm=dm_active)

# modified Fock matrix gives correct result when change of basis used on embedded pyscf obj!!!
g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
g_A = test_full_system_scf.get_veff(dm=dm_active)

v_emb = g_A_and_B - g_A + projector_ortho

In [ ]:
# old mu shift
# projector_mu = mu * (S_mat @ dm_enviro  @ S_mat)
# np.allclose(projector_mu, projector_mu/2)

In [ ]:
Fock_loc_basis = test_full_system_scf.get_fock()
Fock_loc_basis = S_neg_half@Fock_loc_basis @ S_neg_half

if projector_method == 'mu_shfit':
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)

    ## expect subsystem A orbital energies to remain the SAME
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                        full_system_scf.mo_energy[active_MO_inds])} \n''')

    # expect subsystem B orbital energies to be pushed up to high energy
    print(f'original MO energies env: {full_system_scf.mo_energy[enviro_MO_inds]}')
    print(f'new MO energies env: {new_orbital_energies[enviro_MO_inds]}')
    # print(f'''enviro subsystem orbital energies energy increased by mu: 
    #                                                     {np.allclose(new_orbital_energies[enviro_MO_inds],
    #                                                     mu*full_system_scf.mo_energy[enviro_MO_inds])}''')
    
    
elif projector_method == 'huzinaga':
    ## projector should give opposite sign val of subsytem B orbitals vs standard Fock eval
    proj_orb_energy = Loc_Ortho[:, enviro_MO_inds].conj().T @projector_ortho@Loc_Ortho[:, enviro_MO_inds]
    fock_orb_energy = Loc_Ortho[:, enviro_MO_inds].conj().T @Fock_loc_basis@Loc_Ortho[:, enviro_MO_inds]
    
    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    
    new_orbital_energies = np.diag(Loc_Ortho.conj().T @ (Fock_loc_basis+projector_ortho) @ Loc_Ortho)
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                    full_system_scf.mo_energy[active_MO_inds])} \n''')
    
    print(f'''enviro subsystem orbital energies should be ZERO (traced out):{ 
                                   np.allclose(new_orbital_energies[enviro_MO_inds],
                                                np.zeros_like(new_orbital_energies[enviro_MO_inds]))} \n''')
    
else:
    raise ValueError(f'Unknown projection method {projector_method}')


print('\nbasically want active MOs first... then enviroment')
print(f'active MO indices: {active_MO_inds} \n')
print('should appear first when orbital inds listed by energy:')
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

In [ ]:
# TODO ^^^^ choose frozen orbitals here!

In [ ]:
print('Trace(y_ENV @ PB):', np.einsum('ij, ij', dm_enviro, projector) )
print('Trace(y_act @ PB):', np.einsum('ij, ij', dm_active, projector) )
print()
print('Trace(y_ENV @ v_emb):', np.einsum('ij, ij', dm_enviro, v_emb) )
print('Trace(y_act @ v_emb):', np.einsum('ij, ij', dm_active, v_emb) )

In [ ]:
def Get_new_Veff_S_half(pyscf_obj, Unitary, S_neg_half, dm=None, check_result=False):
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    
#     Evaluate RKS/UKS XC functional and potential matrix on given meshgrids
#     for a set of density matrices.
    nelec, exc, vxc = numint.nr_vxc(pyscf_obj.mol,
                                            pyscf_obj.grids,
                                            pyscf_obj.xc,
                                            density_mat)
    vxc =  Unitary.conj().T @ vxc @ Unitary
    vxc =  S_neg_half @ vxc @ S_neg_half # <----------------- NEW
    
    Veff = RKS_get_veff(pyscf_obj, dm=density_mat)
    if Veff.vk is not None:
        K = Unitary.conj().T @ Veff.vk @ Unitary
        J = Unitary.conj().T @ Veff.vj @ Unitary
        vxc += J - K * .5
    else:
        J = Unitary.conj().T @ Veff.vj @ Unitary
        K = None
        vxc += J 
    
    if check_result is True:
        M1 = Unitary.conj().T @ Veff.__array__() @ Unitary
        if not np.allclose(vxc, M1):
            raise ValueError('Veff in new basis NOT correct')
    
    ecoul = np.einsum('ij,ji', density_mat, J).real * .5 # note J matrix is in new basis!
    ## this ecoul term changes if the full density matrix is NOT 
    # (aka for dm_active and dm_enviroment we get different V_eff under different bases!)
    
    
    output = lib.tag_array(vxc, ecoul=ecoul, exc=Veff.exc, vj=J, vk=K)
    return output






full_system_mol_EMBEDDED = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf = scf.RKS(full_system_mol_EMBEDDED)
EMBEDDED_full_system_scf.verbose=1
EMBEDDED_full_system_scf.max_memory= memory
EMBEDDED_full_system_scf.conv_tol = 1e-6
EMBEDDED_full_system_scf.xc = low_level_xc_functional



## include change of basis!
Hcore_std = EMBEDDED_full_system_scf.get_hcore()
EMBEDDED_full_system_scf.get_hcore = lambda *args: S_neg_half@ Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V) @ S_neg_half


EMBEDDED_full_system_scf.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1:  Get_new_Veff_S_half(
                                                                                    EMBEDDED_full_system_scf,
                                                                                        V,
                                                                                        S_neg_half,
                                                                                        dm=dm)

Hcore_rotated = EMBEDDED_full_system_scf.get_hcore()

# overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf.get_hcore = lambda *args: v_emb + Hcore_rotated

E_emb = EMBEDDED_full_system_scf.kernel()

print(f'embedded Energy: {E_emb}')

In [ ]:
EMBEDDED_full_system_scf.mo_energy

In [ ]:
print(f'''number of electrons is correct: {
                                            np.isclose(sum(EMBEDDED_full_system_scf.mo_occ),
                                            2*len(active_MO_inds))
                                          }''')

In [ ]:
C_active_EMBEDDED = EMBEDDED_full_system_scf.mo_coeff[:, EMBEDDED_full_system_scf.mo_occ>0]
dm_active_EMBEDDED = 2* C_active_EMBEDDED @ C_active_EMBEDDED.conj().T

print(EMBEDDED_full_system_scf.energy_elec(dm=dm_active_EMBEDDED)[0])

In [ ]:
# calculate embedding correction term

dm_correction = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED-dm_active)

e_act_emb = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED),
                                       h1e = test_full_system_scf.get_hcore())[0]

# e_act_emb = full_system_scf.energy_elec(dm=dm_active_EMBEDDED,
#                                         vhf= full_system_scf.get_veff(dm=dm_active_EMBEDDED),
#                                        h1e = full_system_scf.get_hcore())[0]

e_mf_emb = e_act_emb + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction
e_mf_emb # <-- energy from embedded DFT calc

In [ ]:
full_system_scf.e_tot

In [ ]:
print(f'''global DFT calculation == seperated calculation: {
                                                            np.isclose(e_mf_emb,
                                                                        full_system_scf.e_tot)
                                                        }''')
# expected as same functional used!

In [ ]:
jkl

# Run more expensive DFT calculation!

In [ ]:
# projector_method ='huzinaga'

In [ ]:
full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )

full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
full_system_mol_HIGH_LEVEL_DFT.build()

full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
full_system_scf_HIGH_LEVEL.verbose=1
full_system_scf_HIGH_LEVEL.max_memory= memory
full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!


# full_system_scf_HIGH_LEVEL.kernel() < --- do NOT RUN THIS


## include change of basis!
Hcore_std = full_system_scf_HIGH_LEVEL.get_hcore()
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
                                                                                    full_system_scf_HIGH_LEVEL, #<-need this here for more expensive xc fucnctional@
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = full_system_scf_HIGH_LEVEL.get_hcore()



if projector_method == 'huzinaga':
#     Fock = (full_system_scf_HIGH_LEVEL.get_hcore() 
#             + full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active + dm_enviro))
    
    Fock = (test_full_system_scf.get_hcore() 
            + test_full_system_scf.get_veff(dm=dm_active + dm_enviro))
    
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    # Huzinaga
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
    projector = S_half @ projector_ortho  @ S_half
    # g_A_and_B = full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active+dm_enviro)
    # g_A = full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active)

    g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
    g_A = full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active)
    
#     g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
#     g_A = full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active)

    v_emb = g_A_and_B - g_A + projector




# overwrite h_core to include embedding term!!!!
full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: v_emb + Hcore_rotated

e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.kernel()
e_act_emb_HIGH_LVL

In [ ]:
full_system_scf_HIGH_LEVEL.mo_energy

In [ ]:
# TODO:
# this code defines the Huzinaga operator using HIGH_level DFT functional (rather than cheap 
# global one as done above)... This may be important for keeping the subsystems localized!



# full_system_mol_HIGH_LEVEL_DFT = gto.Mole(atom= geometry,
#                       basis=basis,
#                        charge=charge,
#                        spin=spin,
#                       )

# full_system_mol_HIGH_LEVEL_DFT.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!
# full_system_mol_HIGH_LEVEL_DFT.build()

# full_system_scf_HIGH_LEVEL = scf.RKS(full_system_mol_HIGH_LEVEL_DFT)
# full_system_scf_HIGH_LEVEL.verbose=1
# full_system_scf_HIGH_LEVEL.max_memory= memory
# full_system_scf_HIGH_LEVEL.conv_tol = 1e-6
# full_system_scf_HIGH_LEVEL.xc = high_level_xc_functional # <-- BETTER functional!





# ## include change of basis!
# Hcore_std = full_system_scf_HIGH_LEVEL.get_hcore()
# full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
#                                                              Unitary_rot=V)


# full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff(
#                                                                                     full_system_scf_HIGH_LEVEL, #<-need this here for more expensive xc fucnctional@
#                                                                                        V,
#                                                                                         dm=dm)

# Hcore_rotated = full_system_scf_HIGH_LEVEL.get_hcore()

# # overwrite h_core to include embedding term!!!!
# #### Get projector using HIGH LEVEL FOCK MATRIX
# Fock = full_system_scf_HIGH_LEVEL.get_hcore() + full_system_scf_HIGH_LEVEL.get_veff(dm=dm_active + dm_enviro)
# Fock_ortho = S_neg_half@Fock @ S_neg_half
# # Huzinaga
# projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
# projector = S_half @ projector_ortho  @ S_half

# v_emb_HIGH_LEVEL = g_A_and_B - g_A + projector

# full_system_scf_HIGH_LEVEL.get_hcore = lambda *args: v_emb_HIGH_LEVEL + Hcore_rotated

# e_act_emb_HIGH_LVL = full_system_scf_HIGH_LEVEL.kernel()
# e_act_emb_HIGH_LVL

In [ ]:
full_system_scf_HIGH_LEVEL.mo_energy

In [ ]:
C_active_EMBEDDED_high_lvl = full_system_scf_HIGH_LEVEL.mo_coeff[:, full_system_scf_HIGH_LEVEL.mo_occ>0]
dm_active_EMBEDDED_high_lvl = 2 * C_active_EMBEDDED_high_lvl @ C_active_EMBEDDED_high_lvl.conj().T

In [ ]:
# calculate embedding correction term

dm_correction_high_lvl = np.einsum('ij, ij', v_emb, dm_active_EMBEDDED_high_lvl-dm_active)

e_act_emb_high_lvl = test_full_system_scf.energy_elec(dm=dm_active_EMBEDDED_high_lvl,
                                        vhf= test_full_system_scf.get_veff(dm=dm_active_EMBEDDED_high_lvl),
                                       h1e = test_full_system_scf.get_hcore())[0]

E_high_lvl_DFT = e_act_emb_high_lvl + E_env + two_e_cross + test_full_system_scf.energy_nuc() + dm_correction_high_lvl
E_high_lvl_DFT # <-- energy from embedded DFT calc

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

In [ ]:
full_system_scf_HIGH_LEVEL.mo_energy

In [ ]:
Fock = (test_full_system_scf.get_hcore() 
        + test_full_system_scf.get_veff(dm=dm_active + dm_enviro))

Fock_ortho = S_neg_half@Fock @ S_neg_half

# Huzinaga
projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!

In [ ]:
PF = projector_ortho @ Fock_ortho
FP = Fock_ortho @ projector_ortho

HUZ = Fock_ortho - PF-FP
# np.allclose(HUZ@projector_ortho, projector_ortho@HUZ)
np.around(HUZ@projector_ortho - projector_ortho@HUZ,7)

In [ ]:
HUZ = S_half @ HUZ  @ S_half

np.around(HUZ@projector - projector@HUZ,7)

In [ ]:
S_DB_S = S_mat @ dm_enviro @ S_mat

fock = test_full_system_scf.get_fock(dm=(dm_enviro + dm_active))
S_DB_F = S_mat @ dm_enviro @ fock
F_DB_S = fock @ dm_enviro @ S_mat

# fock = test_full_system_scf.get_fock(dm=(dm_active))
HUZ = fock - S_DB_F-F_DB_S



np.allclose(HUZ@S_DB_S, S_DB_S@HUZ)

In [ ]:
HUZ@S_DB_S - S_DB_S@HUZ

In [ ]:
# vzcx

# WF hartree-Fock run

In [ ]:
def Get_new_Veff_HFock(pyscf_obj, Unitary, dm=None, check_result=False, hermi=1):
    
    # note this is NOT the same as RKS defintion!
    
    if dm is None:
        if pyscf_obj.mo_coeff is not None:
            density_mat = pyscf_obj.make_rdm1(pyscf_obj.mo_coeff, pyscf_obj.mo_occ)
        else:
            density_mat = pyscf_obj.init_guess_by_1e()
    else:
        density_mat = dm
    
    vj, vk = pyscf_obj.get_jk(dm=density_mat, hermi=hermi)
    Veff = vj - vk * .5
    
#     Veff = pyscf_obj.get_veff(dm=density_mat)
    Veff_new = Unitary.conj().T @ Veff @ Unitary

    return Veff_new

In [ ]:
# define HFock obj (with embedded no. of e-)

full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED_HF) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6

In [ ]:
# Find change of basis for localized orbs
# (NOTE Fock matrix is different with HFock compared to DFT ...
# hence why we recalc it)

S_mat = full_system_scf.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(full_system_mol_EMBEDDED_HF.get_ovlp() , 0.5)

ortho_std = S_half@ full_system_scf.mo_coeff
ortho_loc = S_half@ C_all_localized_and_virt

# ortho_loc[:,0].dot(ortho_loc[:,2])

U = np.zeros((ortho_std.shape[0], ortho_std.shape[0]))
for MO_ind in range(ortho_std.shape[1]):
    outer = np.outer(ortho_std[:, MO_ind], ortho_loc[:, MO_ind])
    U+=outer

print(f'is U*C_ortho_loc =  C_ortho_STD: {(np.allclose(U @ ortho_loc, ortho_std))}')
print(f'is U Udag = Identity: {np.allclose(U.conj().T@U, np.eye(U.shape[0]))}')
                                          
S_neg_half = sp.linalg.fractional_matrix_power(full_system_scf.get_ovlp() , -0.5)

V_chg_B_HFock = S_neg_half @ U @ S_half


print('V @ C_loc = C_std:', np.allclose(V_chg_B_HFock@C_all_localized_and_virt,
                                            full_system_scf.mo_coeff))

In [ ]:
# ############## include change of basis!
Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V_chg_B_HFock)


EMBEDDED_full_system_scf_HF.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
                                                                                    EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
                                                                                       V_chg_B_HFock,
                                                                                        dm=dm)

In [ ]:
## manual projector (in HFock method - different to DFT definition!)

## 1. convert to orthogonal C_matrix
S_mat = EMBEDDED_full_system_scf_HF.get_ovlp()
S_half = sp.linalg.fractional_matrix_power(S_mat , 0.5)
S_neg_half = sp.linalg.fractional_matrix_power(S_mat , -0.5)

Loc_Ortho = S_half@ C_all_localized_and_virt # orthogonal C matrix (localized)

## 2. Define projector that projects MO orbs of subsystem B onto themselves and system A onto zero state!
##### (do this in orthongoal C_matrix!)
### not we only take MO environment indices!
PROJ_ortho = np.zeros_like(S_mat)
for MO_ind in range(C_all_localized_and_virt.shape[1]):
    if MO_ind in enviro_MO_inds:
        outer = np.outer(Loc_Ortho[:, MO_ind], Loc_Ortho[:, MO_ind])
        PROJ_ortho+=outer
    else:
        continue

print(f'''Are subsystem B (env) projected onto themselves: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, enviro_MO_inds], 
        Loc_Ortho[:, enviro_MO_inds])}''') # projected onto itself

print(f'''Is subsystem A traced out?: {
        np.allclose(PROJ_ortho@Loc_Ortho[:, active_MO_inds], 
        np.zeros_like(Loc_Ortho[:, active_MO_inds]))}''') # # projected onto zeros!
        
        
#### 3. NEXT USING PROJECTOR in ortho basis define projector via different methods
        
        
# projector_method = 'huzinaga'
projector_method = 'mu_shfit'

if projector_method == 'huzinaga':
#     # HF fock matrix
    Fock = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active + dm_enviro)
    
    # DFT fock mat
#     Fock = test_full_system_scf.get_hcore() + test_full_system_scf.get_veff(dm=dm_active + dm_enviro)
    
    Fock_ortho = S_neg_half@Fock @ S_neg_half
    
    # Huzinaga
    projector_ortho =  -0.5*(Fock_ortho@PROJ_ortho + PROJ_ortho@Fock_ortho) # 0.5 for restricted calc!
    
    projector = S_half @ projector_ortho  @ S_half
    
elif projector_method == 'mu_shfit':
    mu = 1e6
    projector_ortho = mu * PROJ_ortho
    
#     projector = S_neg_half @ projector_ortho  @ S_half

    # mu shift
    projector = S_half @ projector_ortho  @ S_half
else:
    raise ValueError(f'Unknown projection method {projector_method}')


##################### TODO (start)
# I look at equation 3 of Goodpaster paper
# think we should use DFT potentials (commented out below is Hfock potentials that could be used)
# but projector defined by HFock

# # DFT potentials!
# g_A_and_B = test_full_system_scf.get_veff(dm=dm_active+dm_enviro)
# g_A = test_full_system_scf.get_veff(dm=dm_active)

# # # HFOck potentials!
g_A_and_B = EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active+dm_enviro)
g_A = EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active)

v_emb_HFOCK = g_A_and_B - g_A + projector
##################### TODO (end)

##########
# Fock_loc_basis = EMBEDDED_full_system_scf_HF.get_fock(dm=dm_active+dm_enviro)
Fock_loc_basis = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=dm_active+dm_enviro)

if projector_method == 'mu_shfit':
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)

    ## expect subsystem A orbital energies to remain the SAME
    print('\n')
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                        full_system_scf.mo_energy[active_MO_inds])} ''')
    
    print('actually NOW that we are using HF Fock matix this should NO LONGER MATCH DFT orb energies!!! \n')
    
    # expect subsystem B orbital energies to be pushed up to high energy
    print(f'original MO energies env: {full_system_scf.mo_energy[enviro_MO_inds]}')
    print(f'new MO energies env: {new_orbital_energies[enviro_MO_inds]}')
    # print(f'''enviro subsystem orbital energies energy increased by mu: 
    #                                                     {np.allclose(new_orbital_energies[enviro_MO_inds],
    #                                                     mu*full_system_scf.mo_energy[enviro_MO_inds])}''')
    
    
elif projector_method == 'huzinaga':
    ## projector should give opposite sign val of subsytem B orbitals vs standard Fock eval
    proj_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @projector@C_all_localized_and_virt[:, enviro_MO_inds]
    fock_orb_energy = C_all_localized_and_virt[:, enviro_MO_inds].conj().T @Fock_loc_basis@C_all_localized_and_virt[:, enviro_MO_inds]
    
    print(f''' <env| P |env> = - <env| F |env> : {np.allclose(-1*proj_orb_energy,
                                                              fock_orb_energy)} \n''')
    
    new_orbital_energies = np.diag(C_all_localized_and_virt.conj().T @ (Fock_loc_basis+projector) @ C_all_localized_and_virt)
    
    ## expect subsystem A orbital energies to remain the SAME
    print('\n')
    print(f'''active subsystem orbital energies the same: {np.allclose(new_orbital_energies[active_MO_inds],
                                                        full_system_scf.mo_energy[active_MO_inds])} ''')
    
    print('actually NOW that we are using HF Fock matix this should NO LONGER MATCH DFT orb energies!!! \n')
    
    print(f'''enviro subsystem orbital energies should be ZERO (traced out):{ 
                                   np.allclose(new_orbital_energies[enviro_MO_inds],
                                                np.zeros_like(new_orbital_energies[enviro_MO_inds]))} \n''')
    
else:
    raise ValueError(f'Unknown projection method {projector_method}')


print('\nbasically want active MOs first... then enviroment')
print(f'active MO indices: {active_MO_inds} \n')
print('should appear first when orbital inds listed by energy:')
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

##########



In [ ]:
Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()
## overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb_HFOCK + Hcore_rotated

##############
EMBEDDED_full_system_scf_HF.kernel() 



In [ ]:
print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
print(EMBEDDED_full_system_scf_HF.mo_occ)

C_active_embedd_opt = EMBEDDED_full_system_scf_HF.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0]
HF_dm_mat = 2* C_active_embedd_opt @ C_active_embedd_opt.conj().T

In [ ]:
# emb_orbs = vqe_in_dft.Draw_cube_orbital(full_system_scf,
#                              xyz_string,
#                              EMBEDDED_full_system_scf_HF.mo_coeff, 
#                              list(range(EMBEDDED_full_system_scf_HF.mo_coeff.shape[1])),
#                              width=400, 
#                              height=400, 
#                              jupyter_notebook=True)

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

if projector_method == 'mu_shfit':
    embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
elif projector_method == 'huzinaga':
#     embedded_cc_obj.frozen  = enviro_MO_inds.tolist()
    embedded_cc_obj.frozen  = list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))
else:
    raise ValueError(f'unknown projector method {projector_method}')


# embedded_cc_obj.frozen = None
e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=HF_dm_mat),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf


In [ ]:
embedded_cc_obj.frozen

In [ ]:
WF_correction = np.einsum('ij, ij', v_emb, dm_active)

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 


In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
Fock_emb = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=HF_dm_mat)


print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
orbE = C_active_embedd_opt.conj().T @ Fock_emb @ C_active_embedd_opt
print(np.around(orbE, 7))

np.allclose(EMBEDDED_full_system_scf_HF.mo_energy[:len(active_MO_inds)],
           np.diag(orbE))


In [ ]:
gsdfgsdf

This contains the DFT embedding term (in Veff of Hartree Fock calc)

Whereas in above calc Veff contains v_emb_FOCK (defined using HFock theory!)

DON'T KNOW WHICH ONE IS CORRECT

In [ ]:
full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
                      basis=basis,
                       charge=charge,
                       spin=spin,
                      )
full_system_mol_EMBEDDED_HF.build()

# RE-DEFINE number of electrons in system
full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
EMBEDDED_full_system_scf_HF.verbose=1
EMBEDDED_full_system_scf_HF.max_memory= memory
EMBEDDED_full_system_scf_HF.conv_tol = 1e-6


############## include change of basis!
Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
                                                             Unitary_rot=V)


EMBEDDED_full_system_scf_HF.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
                                                                                    EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
                                                                                       V,
                                                                                        dm=dm)

Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()

## overwrite h_core to include embedding term!!!!
EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb + Hcore_rotated

##############



EMBEDDED_full_system_scf_HF.kernel() 


# # # overwrite orbs with RKS embedded orbs!
# EMBEDDED_full_system_scf_HF.mo_coeff = full_system_scf_HIGH_LEVEL.mo_coeff
# EMBEDDED_full_system_scf_HF.mo_occ = full_system_scf_HIGH_LEVEL.mo_occ 
# EMBEDDED_full_system_scf_HF.mo_energy = full_system_scf_HIGH_LEVEL.mo_energy

# print(EMBEDDED_full_system_scf_HF.energy_elec(dm=dm_active_EMBEDDED_high_lvl)[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot(dm=dm_active_EMBEDDED_high_lvl))
# print()
# print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
EMBEDDED_full_system_scf_HF.mo_energy

In [ ]:
# full_system_mol_EMBEDDED_HF = gto.Mole(atom= geometry,
#                       basis=basis,
#                        charge=charge,
#                        spin=spin,
#                       )
# full_system_mol_EMBEDDED_HF.build()

# # RE-DEFINE number of electrons in system
# full_system_mol_EMBEDDED_HF.nelectron = 2*len(active_MO_inds) # <------ IMPORTANT!

# EMBEDDED_full_system_scf_HF = scf.RHF(full_system_mol_EMBEDDED) # <---- Hartree Fock Calc!
# EMBEDDED_full_system_scf_HF.verbose=1
# EMBEDDED_full_system_scf_HF.max_memory= memory
# EMBEDDED_full_system_scf_HF.conv_tol = 1e-6


# ############## include change of basis!
# Hcore_std = EMBEDDED_full_system_scf_HF.get_hcore()
# EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: Get_new_Hcore(Hcore_std, 
#                                                              Unitary_rot=V)

# # mistake here with HIGH_LEVEL TERM
# # MISTAKE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! (NEED HFOCK OBJECT NOT DFT)
# full_system_scf_HIGH_LEVEL.get_veff = lambda mol=None, dm=None, dm_last=0, vhf_last=0, hermi=1: Get_new_Veff_HFock(
#                                                                                     EMBEDDED_full_system_scf_HF, #<-need this here for more expensive xc fucnctional@
#                                                                                        V,
#                                                                                         dm=dm)

# Hcore_rotated = EMBEDDED_full_system_scf_HF.get_hcore()

# ## overwrite h_core to include embedding term!!!!
# EMBEDDED_full_system_scf_HF.get_hcore = lambda *args: v_emb + Hcore_rotated

# ##############



# EMBEDDED_full_system_scf_HF.kernel() # <------ do NOT RUN!


# # # # overwrite orbs with RKS embedded orbs!
# # EMBEDDED_full_system_scf_HF.mo_coeff = full_system_scf_HIGH_LEVEL.mo_coeff
# # EMBEDDED_full_system_scf_HF.mo_occ = full_system_scf_HIGH_LEVEL.mo_occ 
# # EMBEDDED_full_system_scf_HF.mo_energy = full_system_scf_HIGH_LEVEL.mo_energy

# # print(EMBEDDED_full_system_scf_HF.energy_elec(dm=dm_active_EMBEDDED_high_lvl)[0])
# # print(EMBEDDED_full_system_scf_HF.energy_tot(dm=dm_active_EMBEDDED_high_lvl))
# # print()
# # print(EMBEDDED_full_system_scf_HF.energy_elec()[0])
# # print(EMBEDDED_full_system_scf_HF.energy_tot())

In [ ]:
print(EMBEDDED_full_system_scf_HF.mo_energy)
print(EMBEDDED_full_system_scf_HF.mo_occ)

C_active_embedd_opt = EMBEDDED_full_system_scf_HF.mo_coeff[:,EMBEDDED_full_system_scf_HF.mo_occ>0]
HF_dm_mat = 2* C_active_embedd_opt @ C_active_embedd_opt.conj().T

In [ ]:
emb_Fock = EMBEDDED_full_system_scf_HF.get_hcore() + EMBEDDED_full_system_scf_HF.get_veff(dm=HF_dm_mat)


print(EMBEDDED_full_system_scf_HF.mo_energy)
print()
orbE = C_active_embedd_opt.conj().T @ emb_Fock @ C_active_embedd_opt
print(np.around(orbE, 7))

np.allclose(EMBEDDED_full_system_scf_HF.mo_energy[:len(active_MO_inds)],
           np.diag(orbE))


In [ ]:
# # TODO:

# May have to keep re-defining Huzinaga operator to get rid of those orbitals 
# As I think one iteration of HFock undoes what the projector operator is doing
# Need to check this though (as clearly Huz is defiend by DFT enviroment NOT the optimized one)
# so it could actually be okay!

# Just it is clear from mu shift orbs what is the environement
# not sure if for Huz op the env orbs should be at zero energy! 

In [ ]:
sorted([(i, new_orbital_energies[i])for i in range(new_orbital_energies.shape[0])], key=lambda x:x[1])

In [ ]:
enviro_MO_inds

In [ ]:
## choose frozen from here^^^^
print(enviro_MO_inds)
print(f'for mu shift expect it to be the LAST {len(enviro_MO_inds)}')

print(f'''for Huzinaga expect it to be positions {list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))}''')


In [ ]:
# TODO: Choice of frozen orbs for Huzinaga method could be wrong
# need to CHECK this!

In [ ]:
## CCSD calculation

embedded_cc_obj = cc.CCSD(EMBEDDED_full_system_scf_HF)

if projector_method == 'mu_shfit':
    embedded_cc_obj.frozen = [i for i in range(EMBEDDED_full_system_scf_HF.mol.nao - len(enviro_MO_inds),
                                           EMBEDDED_full_system_scf_HF.mol.nao)
                         ]
elif projector_method == 'huzinaga':
    embedded_cc_obj.frozen  = enviro_MO_inds.tolist()
#     embedded_cc_obj.frozen  = list(range(len(active_MO_inds), len(active_MO_inds)+len(enviro_MO_inds)))
else:
    raise ValueError(f'unknown projector method {projector_method}')


e_cc, t1, t2 = embedded_cc_obj.kernel()

CC_flag_check = np.isclose(EMBEDDED_full_system_scf_HF.energy_tot(dm=HF_dm_mat),
                          embedded_cc_obj.e_hf)


print(f'E_corr: {e_cc}')
print(f'\nCC hartree fock energy matches HF embedded calc: {CC_flag_check}!')
embedded_cc_obj.e_hf


In [ ]:
projector_method

In [ ]:
# print('huzinaga HFock = ', -16.203577904621056)
# print('mu_shfit HFock = ', -16.028522482287855)

In [ ]:
WF_correction = np.einsum('ij, ij', v_emb, dm_active)

E_WF = embedded_cc_obj.e_hf +e_cc  + E_env + two_e_cross - WF_correction
E_WF 

In [ ]:

print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))

print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))

In [ ]:
fkaisdjlfkds

# QC part

In [ ]:
N_enviroment_MOs = len(enviro_MO_inds) 

one_body_integrals, two_body_integrals = vqe_in_dft.Get_embedded_one_and_two_body_integrals_MO_basis(EMBEDDED_full_system_scf_HF,
                                                                                            N_enviroment_MOs,
                                                                                            physists_notation=phys_notation)


print(one_body_integrals.shape)
print(two_body_integrals.shape)

In [ ]:
print(active_MO_inds)
print(enviro_MO_inds)

In [ ]:
one_body_terms, two_body_terms = vqe_in_dft.Get_SpinOrbs_from_Spatial(one_body_integrals,
                                                                   two_body_integrals,
                                                                   physists_notation=phys_notation,
                                                                   EQ_Tolerance=1e-8)


In [ ]:
Nuclear_energy =  full_system_scf.energy_nuc()

H_fermionic = vqe_in_dft.Get_fermionic_H(one_body_terms, 
                                     two_body_terms, 
                                     Nuclear_energy,
                                     core_constant=0, 
                                     physists_notation=phys_notation,
                                        jupyter_notebook=True)



len(list(H_fermionic))

In [ ]:
from openfermion.linalg import get_sparse_operator
H_sparse = get_sparse_operator(H_fermionic)




eigvals_EMBED, eigvecs_EMBED = sp.sparse.linalg.eigsh(H_sparse, which='SA', k=1)

In [ ]:
WF_correction  = np.einsum('ij, ij', v_emb, dm_active) # note different definition

E_VQE = eigvals_EMBED[0]  + E_env + two_e_cross - WF_correction
E_VQE 

In [ ]:
N_electrons_expected = 2*len(active_MO_inds)
N_electrons_Q_state = np.binary_repr(np.where(np.abs(eigvecs_EMBED)>1e-2)[0][0]).count('1') 

print(f'expect {N_electrons_expected} electrons')
print(f'quantum state has {N_electrons_Q_state} electrons \n')

print(f'number of electrons correct: {N_electrons_expected == N_electrons_Q_state}')

In [ ]:
print('High level DFT in DFT error:', np.abs(E_high_lvl_DFT-my_fci.e_tot))
print('LOW level DFT in DFT error:', np.abs(e_mf_emb-my_fci.e_tot))


print('WF in DFT error:', np.abs(E_WF-my_fci.e_tot))
print('VQE error:', np.abs(E_VQE-my_fci.e_tot))